# 所有代码按顺序直接运行即可 此文件中部分代码未显示出结果请看 测试_含部分运行结果代码 文件

### 引入包

In [1]:
import numpy as np
import pandas as pd
import random
import copy
import math
import os

In [2]:
import warnings
warnings.filterwarnings(action = 'ignore',message = "^internal gelsd")

### 导入数据

In [3]:
#已进行过处理，筛选出所有个人用户
download=pd.read_csv("E:\\2020 慧源共享·数据悦读第二届高校开放数据创新研究大赛\\数据1\\下载_个人.csv",error_bad_lines=False)
see=pd.read_csv("E:\\2020 慧源共享·数据悦读第二届高校开放数据创新研究大赛\\数据1\\浏览_个人.csv",error_bad_lines=False)

C:\Users\yuki\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#已进行过处理，导入总分类数据
f=open('总分类.txt','r')
a=f.read()
classfy=eval(a)
f.close()

article_classfy=copy.deepcopy(classfy)

In [5]:
#导入数据清理后的分类数据
f=open('处理好的文章分类.txt','r')
a=f.read()
article_classfy=eval(a)
f.close()
f=open('处理好的文章对应字典.txt','r')
a=f.read()
article_dict=eval(a)
f.close()

In [6]:
#导入用户字典数据
f=open('用户_论文.txt','r')
a=f.read()
user_ratings=eval(a)
f.close()
f=open('用户_论文_次数.txt','r')
a=f.read()
user_ratings_count=eval(a)
f.close()

In [7]:
#导入处理后的用户字典数据
f=open('用户_论文_del.txt','r')
a=f.read()
user_ratings_del=eval(a)
f.close()
f=open('用户_论文_次数_del.txt','r')
a=f.read()
user_ratings_count_del=eval(a)
f.close()

user_ratings_del2=copy.deepcopy(user_ratings_del)
user_ratings_count_del2=copy.deepcopy(user_ratings_count_del)

In [8]:
#导入处理后的按中图分类号给用户分类的字典
f=open('用户_分类号_次数.txt','r')
a=f.read()
user_classcode=eval(a)
f.close()
f=open('分类号_用户.txt','r')
a=f.read()
user_classfy=eval(a)
f.close()

user_have=list(user_ratings_del2.keys())

In [9]:
#导入id字符串转变成序号的字典
f=open('用户_论文_序号.txt','r')
a=f.read()
user_ratings_del2_num=eval(a)
f.close()

### 数据清洗与处理

#### 已提前处理过并存起来的数据的处理方法

In [ ]:
'''#首先定大范围分类来测试推荐准确性，按中图分类号第一个字母进行分类
#首先看此文章对应的中图分类号的长度，大于一的话可能需要归为多类
classfy={}
for aid in article_dict.keys():
    num=len(article_dict[aid])#长度
    if num==1:
        code=article_dict[aid][0][0]
        if code in classfy:
            if aid not in classfy[code]:
                classfy[code].append(aid)
        else:
            classfy[code]=[]
            classfy[code].append(aid)
    else:
        for i in range(0,num):
            code=article_dict[aid][i][0]
            if code in classfy:
                if aid not in classfy[code]:
                    classfy[code].append(aid)
            else:
                classfy[code]=[]
                classfy[code].append(aid)       '''

'''#存储
#生成的字典存成txt的方法
f=open('总分类.txt','w')
f.write(str(classfy))
f.close()'''

In [ ]:
'''#把类别错误的文献删去

#首先把分类号开头是一个空格的文献的空格删去
a_id=article_classfy[' ']
index=article_list.index(a_id[0])
article_dict[a_id[0]][1]=article_dict[a_id[0]][1][1:]
article_classfy[article_dict[a_id[0]][1][0]].append(a_id[0])
del article_classfy[' ']

classcode[index]=article_dict[a_id[0]][0]+';'+article_dict[a_id[0]][1]


#把分类号开头是中文的前面几个中文删去
a_id=article_classfy['中']
index=article_list.index(a_id[0])
article_dict[a_id[0]][0]=article_dict[a_id[0]][0][7:]
article_classfy[article_dict[a_id[0]][0][0]].append(a_id[0])
del article_classfy['中']

classcode[index]=article_dict[a_id[0]][0]


#把分类号开头数字或其他符号的论文删去
nega_code=[]
for code in article_classfy.keys():
    if code.isalpha():
        #当确定为字母时。全部转化为大写字母
        if code.islower():
            code_to_upper=code.upper()
            ar=article_classfy[code]
            nega_code.append(code)
            for a_c in ar:
                article_classfy[code_to_upper].append(a_c)
                
            #在article_dict中找到相应的图文分类号，转化成大写字母
            a_id=article_classfy[code]
            for i in a_id:
                i_code=article_dict[i]
                for i_c in range(0,len(i_code)):
                    if i_code[i_c][0].islower():
                        article_dict[i][i_c]=article_dict[i][i_c][0].upper()+article_dict[i][i_c][1:]
                        
    else:
        #当不是字母时,从article_classfy article_dict中删除
        #提取出所有不符合条件的论文名
        a_id=article_classfy[code]
        #把这些论文中不符合条件的图文分类号从dict中删除
        for a in a_id:
            #检查dict中是否还有别的分类号
            co=article_dict[a]
            if len(co)>1:
                for c in co:
                    if c[0]==code:
                        article_dict[a].remove(c)
            else:
                del article_dict[a]
        #删除classfy中不符合条件的分类号项
        nega_code.append(code)
        
for nega in nega_code:
    del article_classfy[nega]
    
#存储处理好的文章分类
#生成的字典存成txt的方法
f=open('处理好的文章分类.txt','w')
f.write(str(article_classfy))
f.close()
f=open('处理好的文章对应字典.txt','w')
f.write(str(article_dict))
f.close()'''

In [ ]:
'''#建立用户：互动文章字典
user_ratings={}  #普通的用户：互动文章 字典
user_ratings_count={}    #用户：[互动文章][对应次数] 字典
I=range(temp_all_u_a.index.size)
for i in I:
    user_id=temp_all_u_a.loc[i,'user_id']
    article_id=temp_all_u_a.loc[i,'article_id']
    count=temp_all_u_a.loc[i,'count']
    
    if user_id in user_ratings.keys():
        if article_id in user_ratings[user_id]:
            continue
        else:
            user_ratings[user_id].append(article_id)
            user_ratings_count[user_id]['article'].append(article_id)
            user_ratings_count[user_id]['count'].append(count)
    else:
        user_ratings[user_id]=[]
        user_ratings_count[user_id]={}
        user_ratings_count[user_id]['article']=[]
        user_ratings_count[user_id]['count']=[]
        user_ratings[user_id].append(article_id)
        user_ratings_count[user_id]['article'].append(article_id)
        user_ratings_count[user_id]['count'].append(count)
        '''

'''#字典存储
#生成的字典存成txt的方法
f=open('用户_论文.txt','w')
f.write(str(user_ratings))
f.close()
f=open('用户_论文_次数.txt','w')
f.write(str(user_ratings_count))
f.close()'''

In [ ]:
'''#提取所有有中图分类号的论文
article_have=list(article_dict.keys())

#无中图分类号的论文从用户字典中删去
user_ratings_del={}
user_ratings_count_del={}
for user in user_ratings.keys():
    index=user_ratings[user]
    count=user_ratings_count[user]['count']
    user_ratings_del[user]=[]
    user_ratings_count_del[user]={}
    user_ratings_count_del[user]['article']=[]
    user_ratings_count_del[user]['count']=[]
    for i in range(0,len(index)):
        if index[i] in article_have:
            user_ratings_del[user].append(index[i])
            user_ratings_count_del[user]['article'].append(index[i])
            user_ratings_count_del[user]['count'].append(count[i])
            
            
#字典存储
#生成的字典存成txt的方法
f=open('用户_论文_del.txt','w')
f.write(str(user_ratings_del))
f.close()
f=open('用户_论文_次数_del.txt','w')
f.write(str(user_ratings_count_del))
f.close()'''

In [ ]:
'''#按中图分类号的给用户分类
user_classfy={}
user_classcode={}
for user in user_ratings_del2.keys():
    article=user_ratings_del2[user]
    count=user_ratings_count_del2[user]['count']
    code=[]
    for idd in article:
        a_code=article_dict[idd]
        t=[]
        for c in a_code:
            t.append(c[0])
        code.append(t)
    
    #记录偏爱的classcode类的比例
    user_classcode[user]={}
    for cc in code:
        if type(cc)==list:
            for c in cc:
                if c not in user_classcode[user].keys():
                    user_classcode[user][c]=0
                    user_classcode[user][c]=count[code.index(cc)]
                else:
                    user_classcode[user][c]+=count[code.index(cc)]
        else:
            if cc not in user_classcode[user].keys():
                user_classcode[user][cc]=0
                user_classcode[user][cc]=count[code.index(cc)]
            else:
                user_classcode[user][cc]+=count[code.index(cc)]
    c1=[]
    for cc in code:
        if type(cc)==list:
            for c in cc:
                c1.append(c)
        else:
            c1.append(cc)
    code=list(set(c1))
    for co in code:
        if co in user_classfy.keys():
            if user not in user_classfy[co]:
                user_classfy[co].append(user)
        else:
            user_classfy[co]=[]
            user_classfy[co].append(user)
            
            
#通过观察user_classfy可以得出 还是存在部分未清理完全的数据 进行单独清理
user=user_classfy['3']
a1=user_ratings[user[0]]
a2=user_ratings[user[1]]
a3=user_ratings[user[2]]
for a in a1:
    co=article_dict[a]
    for c in co:
        if c[0]=='3':
            print(a)
            print(c)
print('next')
for a in a2:
    co=article_dict[a]
    for c in co:
        if c[0]=='3':
            print(a)
            print(c)
print('next')
for a in a3:
    if a in article_dict.keys():
        co=article_dict[a]
        for c in co:
            if c[0]=='3':
                print(a)
                print(c)
                
                
article_dict['HY000001473620'][1]=article_dict['HY000001473620'][1][2:]
article_dict['HY000001473620'][2]=article_dict['HY000001473620'][2][2:]
article_dict['HY000001473620'][3]=article_dict['HY000001473620'][3][2:]

article_dict['HY000001609650'][1]=article_dict['HY000001609650'][1][2:]
article_dict['HY000001609650'][2]=article_dict['HY000001609650'][2][2:]

article_dict['HY000001657044'][1]=article_dict['HY000001657044'][1][2:]
article_dict['HY000001657044'][2]=article_dict['HY000001657044'][2][2:]

article_dict['HY000002163199'][1]=article_dict['HY000002163199'][1][2:]
article_dict['HY000002163199'][2]=article_dict['HY000002163199'][2][2:]

article_dict['HY000002221331'][1]=article_dict['HY000002221331'][1][2:]
article_dict['HY000002221331'][2]=article_dict['HY000002221331'][2][2:]

article_dict['HY000002314115'][1]=article_dict['HY000002314115'][1][2:]
article_dict['HY000002314115'][2]=article_dict['HY000002314115'][2][2:]


#重新运行按中图分类号的给用户分类的那一段代码

#字典存储
#生成的字典存成txt的方法
f=open('用户_分类号_次数.txt','w')
f.write(str(user_classcode))
f.close()
f=open('分类号_用户.txt','w')
f.write(str(user_classfy))
f.close()'''

In [ ]:
'''#构造把各自id转变成列表中序号的字典
#论文字典article_have
#用户字典user_have
user_ratings_del2_num={}
for user in user_ratings_del2.keys():
    index_user=user_have.index(user)
    user_ratings_del2_num[index_user]=[]
    arti=user_ratings_del2[user]
    for i in range(0,len(arti)):
        index_article=article_have.index(arti[i])
        user_ratings_del2_num[index_user].append(index_article)
        
        
#字典存储
#生成的字典存成txt的方法
f=open('用户_论文_序号.txt','w')
f.write(str(user_ratings_del2_num))
f.close()'''

#### 处理数据

In [10]:
download=download.reset_index()
see=see.reset_index()

In [11]:
article=set(list(download['article_id'].values)+list(see['article_id'].values))
article_id=list(article)
article_id=article_id[1:]
article_id.sort(reverse=False)

In [12]:
down_article_classcode=download.drop(columns=['user_id','user_type','date_time','article_title','keywords','author','unit','type','province'])
see_article_classcode=see.drop(columns=['user_id','user_type','date_time','article_title','keywords','author','unit','type','province'])

In [13]:
#去重
down_a_c=down_article_classcode.groupby(['article_id','classcode']).size().reset_index(name='count')
see_a_c=see_article_classcode.groupby(['article_id','classcode']).size().reset_index(name='count')

#两个数据框合到一起
test_down=down_a_c.drop(columns=['count'])
test_see=see_a_c.drop(columns=['count'])
test=pd.concat([test_down,test_see])

#去重
test_set=test.groupby(['article_id','classcode']).size().reset_index(name='count')

In [14]:
#article_list和classcode提取出来建立列表
article_list=list(test_set['article_id'].values)
classcode=list(test_set['classcode'].values)

In [15]:
#检查是否有中图分类号形式不正确,进行部分数据清理
for i in range(0,len(classcode)-1):
    if "\n" in classcode[i]:
        classcode[i]=classcode[i][:6]+classcode[i][-7:-1]
        

#制作文献名对应拆开的中图分类号的字典
article_dict={}
for i in range(0,len(classcode)):
    if ';' in classcode[i]:
        code=classcode[i].split(';')
    else:
        code=[classcode[i]]
    article_dict[article_list[i]]=code
    
#检查是否有空字符单独成为列表中的某个元素,删除空字符元素
for aid in article_dict.keys():
    if '' in article_dict[aid]:
        article_dict[aid].remove('')

In [16]:
#制作每个类别对应的用户字典

users=set(list(download['user_id'].values)+list(see['user_id'].values))
users_list=list(users)
users_list.sort(reverse=False)

temp_down_u_a=download.drop(columns=['user_type','date_time','article_title','keywords','author','unit','type','province','classcode'])
temp_see_u_a=see.drop(columns=['user_type','date_time','article_title','keywords','author','unit','type','province','classcode'])

temp_down_u_a=temp_down_u_a.groupby(['user_id','article_id']).size().reset_index(name='count')
temp_see_u_a=temp_see_u_a.groupby(['user_id','article_id']).size().reset_index(name='count')

t1=temp_down_u_a.drop(columns=['count'])
t2=temp_see_u_a.drop(columns=['count'])
#合并
temp_all_u_a=pd.concat([t1,t2])
#去重
temp_all_u_a=temp_all_u_a.groupby(['user_id','article_id']).size().reset_index(name='count')

In [17]:
user_not_0=[]
user_not_1=[]
for user in user_ratings_del2.keys():
    index=user_ratings_del2[user]
    num=len(index)
    if num==0:
        user_not_0.append(user)
    elif num==1:
        user_not_1.append(user)
        
#删去只跟一个或零个有分类号的论文发生过关系的用户
for user in user_not_0:
    del user_ratings_del2[user]
    del user_ratings_count_del2[user]
for user in user_not_1:
    del user_ratings_del2[user]
    del user_ratings_count_del2[user]

### 引入带有时间机制的模型计算过程

In [ ]:
'''#读取含时间的数据

globals = {
    'nan': 0   
}

#存成txt的文件重新生成字典的方法
f=open('E:\\2020 慧源共享·数据悦读第二届高校开放数据创新研究大赛\\data\\info_user.txt','r')
a=f.read()
dic=eval(a,globals)
f.close()

dic2=copy.deepcopy(dic)

for user in users_list:
    ar_list=list(dic2[user].keys())
    for ar in ar_list:
        if ar not in article_have:
            del dic2[user][ar]
            
#去除无满足条件论文的用户
for user in users_list:
    if len(dic2[user])==0:
        del dic2[user]
        
#去除只互动一次的用户，保证计算出的用户与user_ratings_del2中的用户一致
u_t=[]
for user in dic2.keys():
    if user not in user_ratings_del2:
        u_t.append(user)
        
for u in u_t:
    del dic2[u]
    
#字典存储
#生成的字典存成txt的方法
f=open('info_dic2.txt','w')
f.write(str(dic2))
f.close()'''

In [18]:
globals = {
    'nan': 0   
}
f=open('info_dic2.txt','r')
a=f.read()
dic2=eval(a,globals)
f.close()

#### 进行相应算法处理

In [19]:
import datetime
def date_subtraction(date_lis,time_str2):
    res = []
    length = len(date_lis)
    for i in range(length):
        date_str1 = date_lis[i][:10]
        date_str2 = time_str2[:10]
        date1 = datetime.datetime.strptime(date_str1,'%Y-%m-%d').date()
        date2 = datetime.datetime.strptime(date_str2,'%Y-%m-%d').date()
        subtraction = (date2 - date1)
        
        if subtraction.days < 0 :
            print('相差时间为负值，已取得觉绝对值')
            res.append(-subtraction.days)
        else:
            res.append(subtraction.days)

    return(res)

max_num = 3
make_inf = 300000
import time
today = time.strftime("%Y-%m-%d",time.localtime(time.time()))########################日期，设置自动更新了
def OP_essay(user_id,essay_id,max_num = 3,make_inf = 300000):
    #reward_down = min(dic[user_id][essay_id]['下载_看_次数'][0],max_num)
    #reward_see = min(dic[user_id][essay_id]['下载_看_次数'][1],max_num)
    reward_down = dic2[user_id][essay_id]['下载_看_次数'][0]
    reward_see = dic2[user_id][essay_id]['下载_看_次数'][1]
    # 浏览是否转换成下载
    reward_turn = 0
    if dic2[user_id][essay_id]['下载_看_次数'][0] != 0 and dic2[user_id][essay_id]['下载_看_次数'][1] !=0:
        reward_turn = 1
    # 下载：时间衰减机制
    temp = dic2[user_id][essay_id]['下载_看_时间'][0]
    down_subtraction = [make_inf]
    if temp:
        down_subtraction = date_subtraction(dic2[user_id][essay_id]['下载_看_时间'][0],today)
        
    # 浏览：时间衰减机制
    temp = dic2[user_id][essay_id]['下载_看_时间'][1]
    see_subtraction = [make_inf]
    if temp:
        see_subtraction = date_subtraction(dic2[user_id][essay_id]['下载_看_时间'][1],today)
        
    return([reward_down,reward_see,reward_turn,down_subtraction,see_subtraction])

def OP_user(user_id,max_num):
    data = []
    for user_id in dic2.keys():
        for essay_id in dic2[user_id].keys():
            data.append(OP_essay(user_id,essay_id,max_num = 3))
    return(data)


data = {'user_id':[],'essay_id':[],'reward_down':[],'reward_see':[],'reward_turn':[],'down_subtraction':[],'see_subtraction':[]}

for user_id in list(dic2.keys()):
    for essay_id in list(dic2[user_id].keys()):
        res = OP_essay(user_id,essay_id,max_num = 3,make_inf = 300000)
        data['user_id'].append(user_id)
        data['essay_id'].append(essay_id)
        data['reward_down'].append(res[0])
        data['reward_see'].append(res[1])
        data['reward_turn'].append(res[2])
        data['down_subtraction'].append(res[3])
        data['see_subtraction'].append(res[4])
 #reward_day = math.exp(-w_time*down_subtraction)

In [20]:
data = pd.DataFrame.from_dict(data)

order = ['user_id', 'essay_id', 'reward_down', 'reward_see', 'reward_turn', 'down_subtraction','see_subtraction']
data = data[order]

#### 综合评价方法 独立性权系数法

In [21]:
# 时间机制的冷却因子
Cooling_factor = 0.6

from sklearn import preprocessing
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

def autoNorm(data):
    """
    :param data: 列表
    :return: 归一化列表
    """
    arr = np.asarray(data)
    norm_list=[]
    for x in arr:
        x = round(float(x - np.min(arr)) / ((np.max(arr) - np.min(arr))+0.001),4)
        norm_list.append(x)

    return norm_list



# 自定义max-min 归一化函数
def min_max_scaler(data):
    """
    :param data: 输入各评价指标的数据框
    :return: 归一化之后的数据框
    """
    X = np.array(data)
    min_max_scaler = preprocessing.MinMaxScaler()
    X_minMax = min_max_scaler.fit_transform(X)
    X_minMax=pd.DataFrame(X_minMax,columns=data.columns.values)
    return X_minMax



def multipl(a, b):
    """
    :param a: 列表1
    :param b: 列表2
    :return: 相乘求和之后的值
    """
    sumofab = 0.0
    for i in range(len(a)):
        temp = a[i] * b[i]
        sumofab += temp
    return sumofab




def corrcoef(x, y):
    """
    :param x: 列表1
    :param y: 列表2
    :return: 复相关系数R
    """
    n = len(x)
    # 求和
    sum1 = sum(x)
    sum2 = sum(y)
    # 求乘积之和
    sumofxy = multipl(x, y)
    # 求平方和
    sumofx2 = sum([pow(i, 2) for i in x])
    sumofy2 = sum([pow(j, 2) for j in y])
    num = sumofxy - (float(sum1) * float(sum2) / n)
    # 计算复相关系数
    den = math.sqrt((sumofx2 - float(sum1 ** 2) / n) * (sumofy2 - float(sum2 ** 2) / n))
    R=num / den
    return R


def get_R_coefficient(data):
    """
    :param data: 评价指标的数据框
    :return:评价指标的复相关系数
    """
    R_list=[]
    for i in range(0,len(data.columns.values)):
        columns_list = list(data.columns.values)
        k1 = columns_list[i]
        columns_list.remove(k1)
        X = sm.add_constant(data.loc[:,columns_list])
        y = data.loc[:,k1]
        est = sm.OLS(y.astype(float), X.astype(float))
        est2 = est.fit()
        predict = est2.predict()
        R=corrcoef(y, predict)
        R_list.append(R)

    return R_list


def get_R_wi(R_list):
    """
    :param R_list: 复相关系数列表
    :return: 各指标的权重
    """
    wi_1=[]
    wi_list=[]
    for i in range(0,len(R_list)):
        # 求各指标与其他指标复相关系数的倒数
        w1=round(1/R_list[i],2)
        wi_1.append(w1)

    # 对上表数据进行归一化处理，即可得到各指标的权重
    sum_wi_1=sum(wi_1)
    for i in range(0,len(wi_1)):
        # w2 = wi_1[i] / sum_wi_1
        w2=round(wi_1[i]/sum_wi_1,2)
        wi_list.append(w2)


    return wi_list

one_year = 365
Cooling_factor = 0.6
def operate_date(train_data,Cooling_factor):
    length = len(train_data)
    for i in range(length):

         # 下载时间机制处理
        length2 = len(data_test['down_subtraction'][i])
        down_subtraction = []

        for j in range(length2):
            down_subtraction.append(math.exp(-Cooling_factor*data_test['down_subtraction'][i][j]/one_year))
        train_data[i,0] = sum(down_subtraction)
        
        
        #浏览事件机制处理
        length3 = len(data_test['see_subtraction'][i])
        
        
        see_subtraction = []
        
        for p in range(length3):
            see_subtraction.append(math.exp(-Cooling_factor*data_test['see_subtraction'][i][p]/one_year))
        train_data[i,1] = sum(see_subtraction)
      
    return(train_data)


def get_article_score(wi_list,data):
    """
    :param wi_list: 权重系数列表
    :param data：评价指标数据框
    :return:返回文章质量得分
    """

    #  将权重转换为矩阵

    cof_var = np.mat(wi_list)

    #  将数据框转换为矩阵
    context_train_data = np.mat(data)
    
    #  权重跟自变量相乘  
    # 自变量：context_train_data
    # 权重：cof_var
    context_train_data = operate_date(context_train_data,Cooling_factor=0.6)
    
    last_hot_matrix = context_train_data * cof_var.T
    
    
    last_hot_matrix = pd.DataFrame(last_hot_matrix.T)
    
    
    #  累加求和得到总分
    last_hot_score = list(last_hot_matrix.apply(sum))

    #  max-min 归一化

    last_hot_score_autoNorm = autoNorm(last_hot_score)

    # 值映射成分数（0-100分）？？？？？

    last_hot_score_result = [i * 1000 for i in last_hot_score_autoNorm]

    return last_hot_score_result

In [22]:
data_test = data.copy(deep=True)

In [23]:
if __name__ == '__main__':
    ##改成小数据
    context_train_data = data.iloc[:,2:5]
    R_list = get_R_coefficient(context_train_data)
    wi_list = get_R_wi(R_list)
    wi_list[2]
    
    last_hot_score_result = get_article_score(wi_list,context_train_data)

    # 增加一列id
    context_train_data['user_id'] = data['user_id']
    context_train_data['essay_id'] = data['essay_id']
    

    # 增加一列分数值
    context_train_data['score'] = last_hot_score_result

    # 然后对数据框从大到小排序

    #result=context_train_data.sort_values(by = 'score',axis = 0,ascending = False)

    #result['rank']=range(1,len(result)+1)

    #print(context_train_data)

## BPR模型

### 建立函数

#### 构造测试集

In [24]:
def generate_test(user_ratings):
    """
    对每一个用户u，在user_ratings中随机找到他评分过的一部电影i,保存在user_ratings_test，我们为每个用户取出的这一个电影，是不会在训练集中训练到的，作为测试集用。
    """
    user_test = dict()
    for u,i_list in user_ratings.items():
        user_test[u] = random.sample(user_ratings[u],1)[0]
    return user_test

In [25]:
def generate_test_batch(user_ratings,user_ratings_test,item_count):
    """
    对于每个用户u，它的评分电影i是我们在user_ratings_test中随机抽取的，它的j是用户u部分没有评分过的论文集合，
    比如用户有n个评分过的论文，那么就从互动过的分类号中随机抽取n个未评分过的论文作为j
    """
    for u in user_ratings.keys():
        #查看所有互动过的分类号
        code=list(user_classcode[user_have[u]].keys())
        #从这些分类号中提取出所有论文
        ar=[]
        for co in code:
            ar=ar+article_classfy[co]
            

        num=len(user_ratings[u])
        t = []
        i = user_ratings_test[u]
        for count in range(num):
            j_id = random.randint(1,len(ar)-1)
            j=article_have.index(ar[j_id])
            if not(j in user_ratings[u]):
                t.append([u,i,j])
            else:
                count-=1
        yield np.asarray(t)

#### 构造训练数据

In [26]:
def generate_train_batch(user_ratings,user_ratings_test,item_count,batch_size=512):
    """
    构造训练用的三元组
    三元组的组成 (u,i,未交互论文j)或(u,分数较高的i,分数较低的i)
    对于随机抽出的用户u，i可以从user_ratings随机抽出，而j也是从总的论文集中随机抽出
    当然j必须保证(u,j)不在user_ratings中
    """
    t = []
    for b in range(batch_size):
        u = random.sample(user_ratings.keys(),1)[0]
        
        #从dic2中提取出所有论文的排名
        rank_index=context_train_data[(context_train_data['user_id'].values==user_have[u])].index.tolist()
        rank_ar=list(context_train_data.loc[rank_index,'essay_id'])
        rank_score=list(context_train_data.loc[rank_index,'score'])
        
        i = random.sample(user_ratings[u],1)[0]
        #test数据集里的数据不能作为训练使用，所以要另外取新值
        while i==user_ratings_test[u]:
            i = random.sample(user_ratings[u],1)[0]

        j = random.randint(1,item_count)
        while j in user_ratings[u]:
            loc1=rank_ar.index(article_have[j])
            loc2=rank_ar.index(article_have[i])
            if rank_score[loc1]<rank_score[loc2]:
                break
            else:
                j = random.randint(1,item_count)

        t.append([u,i,j])

    return np.asarray(t)

#### 构造算法模型

In [27]:
def bpr_mf(user_count,item_count,hidden_dim):
    #hidden_dim是我们矩阵分解的隐含维度k
    
    #预先给三个参数赋予占位，分配必要的内存，等运行模型的时候通过feed_dict喂入数据
    u = tf.placeholder(tf.int32,[None])
    i = tf.placeholder(tf.int32,[None])
    j = tf.placeholder(tf.int32,[None])

    #创建新的tensorflow遍历，shape为count+1到hiddendim
    #创建的是 正态分布初始化器 均值0 方差0.1
    user_emb_w = tf.get_variable("user_emb_w", [user_count + 1, hidden_dim],
                                 initializer=tf.random_normal_initializer(0, 0.1))
    item_emb_w = tf.get_variable("item_emb_w", [item_count + 1, hidden_dim],
                                 initializer=tf.random_normal_initializer(0, 0.1))

    #选取一个张量里面索引对应的元素
    #u,i,j是索引,user_emb_w等是张量
    u_emb = tf.nn.embedding_lookup(user_emb_w, u)
    i_emb = tf.nn.embedding_lookup(item_emb_w, i)
    j_emb = tf.nn.embedding_lookup(item_emb_w, j)

    #MF predict:u_i>u_j
    #multiply 相同位置的元素相乘
    #reduce_sum 计算张量沿着某一维度的和
    #keepdims是否保持原有张量的维度
    x = tf.reduce_sum(tf.multiply(u_emb,(i_emb-j_emb)),1,keep_dims=True)

    
    #每个用户的AUC（AUC是指ROC曲线下的面积，TP越大，ROC曲线越上凸，算法分类效果越优秀）
    #average AUC = mean( auc for each user in test set)
    mf_auc = tf.reduce_mean(tf.to_float(x>0))

    #add_n 列表的元素相加
    l2_norm = tf.add_n([
        tf.reduce_sum(tf.multiply(u_emb, u_emb)),
        tf.reduce_sum(tf.multiply(i_emb, i_emb)),
        tf.reduce_sum(tf.multiply(j_emb, j_emb))
    ])

    #损失函数
    regulation_rate = 0.0001
    bprloss = regulation_rate * l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(x)))

    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(bprloss)
    return u, i, j, mf_auc, bprloss, train_op

### 运行模型

In [28]:
user_ratings_test = generate_test(user_ratings_del2_num)

users_temp=[]
for u in user_ratings_del2_num:
    users_temp.append(u)

In [29]:
#整理代码中暂不运行此代码，若看结果请看测试代码此部分结果

In [ ]:
tf.reset_default_graph()
with tf.Session() as sess:
    u,i,j,mf_auc,bprloss,train_op = bpr_mf(len(users_temp),len(article_have),20)
    sess.run(tf.global_variables_initializer())

    for epoch in range(1,5):
        _batch_bprloss = 0
        for k in range(1,10):# uniform samples from training set5000
            uij = generate_train_batch(user_ratings_del2_num,user_ratings_test,len(article_have))
            
            _bprloss,_train_op = sess.run([bprloss,train_op],
                                          feed_dict={u:uij[:,0],i:uij[:,1],j:uij[:,2]})
            
            _batch_bprloss += _bprloss

        print("epoch:",epoch)
        print("bpr_loss:",_batch_bprloss / k)
        print("_train_op")

        user_count = 0
        _auc_sum = 0.0

        # each batch will return only one user's auc
        for t_uij in generate_test_batch(user_ratings_del2_num, user_ratings_test, len(article_have)):
            _auc, _test_bprloss = sess.run([mf_auc, bprloss],
                                              feed_dict={u: t_uij[:, 0], i: t_uij[:, 1], j: t_uij[:, 2]}
                                              )

            user_count += 1
            _auc_sum += _auc
        print("test_loss: ", _test_bprloss, "test_auc: ", _auc_sum / user_count)
        print("")
    variable_names = [v.name for v in tf.trainable_variables()]
    values = sess.run(variable_names)
    for k, v in zip(variable_names, values):
        print("Variable: ", k)
        print("Shape: ", v.shape)
        print(v)

#这里取了k=20，迭代次数4
#输出的W,H矩阵分别在values[0]和values[1]中。

### 利用计算出的矩阵计算用户的推荐

In [ ]:
#示例 测试用户为 '02412b4ddc1148d2'
user_id='02412b4ddc1148d2'
#查询用户名所对应索引
index=[user_have.index(user_id)]

#查询这个用户推荐排列前五的文献

#用户对这个用户对所有电影的预测评分
session1 = tf.Session()
#expand_dims扩大指定维数 原来values[0][0]为（20，）的数组 变为(1,20)
u1_dim = tf.expand_dims(values[0][index[0]], 0)

#matmul a、b矩阵相乘 transpose_b：如果 True,b 在乘法之前转置
u1_all = tf.matmul(u1_dim, values[1],transpose_b=True)

#计算用户对所有商品的排序
result_1 = session1.run(u1_all)

#squeeze 将结果矩阵的第一维去掉，去掉外面一个框
p = np.squeeze(result_1)

#argsort 返回数组值从小到大的索引值
#把大小除了最大五个以外的p里的其他索引对应的值都设为0
p[np.argsort(p)[:-5000]] = 0

#遍历p中的全部索引，只要值不为0，就输出索引和对应的索引值
article_rec=[]
score=[]
for index in range(len(p)):
    if p[index] != 0:
        score.append([index,p[index]])
        article_rec.append(article_have[index])
        
#转为数据框
score_df=pd.DataFrame(score)

#按顺序进行排序
score_df2=score_df.sort_values(by=1, ascending=False)

#筛选出这个用户感兴趣的分类号，四舍五入按比例选出前200个论文
co=list(user_classcode['02412b4ddc1148d2'].keys())
co_sum=np.sum(list(user_classcode['02412b4ddc1148d2'].values()))
co_wi=[]
for c in co:
    co_wi.append(int(round(user_classcode['02412b4ddc1148d2'][c]/co_sum)))
ar_incode=[]
for c in co:
    flag=0
    for ar_index in list(score_df2[0].values):
        if article_have[ar_index] in article_classfy[c]:
            ar_incode.append(article_have[ar_index])
            flag+=1
            if flag>=200:
                break

In [30]:
#在测试过程中推荐的200个文献列表
f=open('对于某用户的推荐前200位.txt','r')
a=f.read()
ar_incode=eval(a)
f.close()

In [31]:
ar_incode

['ddyylc201801180',
 'tjzyxyxb201601007',
 'gwyx-lcsw201916022',
 'zdxllysj201902003',
 'xtyx201710015',
 'zgetbjzz200201024',
 'Y2257638',
 'zgyyzn201708202',
 'zfysjjbzz201603010',
 'sczlfz200602012',
 'xzy200611021',
 'ylsbxx201907020',
 'xngfyy201901018',
 'ddhs-xsb201901048',
 'syxnfxgbzz201903009',
 'jslcyxzz201613048',
 'zhqkyszz200704022',
 'jtbj201923286',
 'D01297221',
 'Y2248365',
 'gdyx201605030',
 'zgbjyy-kp201703250',
 'zghlgl201901027',
 'yiyqy201731086',
 'nmgyxyxb201905005',
 '7692269',
 'zgazbxb201704011',
 'xbyx201310011',
 'tjyy201601026',
 'sxhlzz201914032',
 'D784482',
 'jxyy201603025',
 'qlhlzz201011056',
 'yxzs201901030',
 'zgxdyxzz201908028',
 'lcxhbzz200202021',
 'zglcsyyx201506017',
 'yxfwyyj201803022',
 '7407212',
 'szybj201922019',
 'Y3119709',
 'zjzyxyxb200901080',
 'hyyjk-syzl201412053',
 'hngfyxzz201703021',
 'njbdyy201803019',
 'dsjydxxb201422003',
 'cqyx201335014',
 'zgbjyy-kp201624113',
 'zgkjzh201513159',
 'zgbjyy-kp201936032',
 'zhzl201503007',
 'sh

In [ ]:
#制作便于查找论文相关信息的数据框
down_article_df=download.drop(columns=['user_id','user_type','date_time','keywords','author','unit','type','province'])
see_article_df=see.drop(columns=['user_id','user_type','date_time','keywords','author','unit','type','province'])
#合并
all_article_df=pd.concat([down_article_df,see_article_df])
#去重
all_article_df=all_article_df.groupby(['article_id','article_title','classcode']).size().reset_index(name='count')

In [ ]:
#挑选排列前15个文献，存储她们的各信息
recom_article_id=[]
recom_article_classcode=[]
recom_article_title=[]
recom_score=[]
for ar in ar_incode[0:15]:
    if ar in all_article_df['article_id'].values:
        recom_article_id.append(ar)
        recom_article_classcode.append(all_article_df.loc[all_article_df[(all_article_df['article_id'].values==ar)].index.tolist()[0],'classcode'])
        recom_article_title.append(all_article_df.loc[all_article_df[(all_article_df['article_id'].values==ar)].index.tolist()[0],'article_title'])
        for i in score:
            if article_have.index(ar)==i[0]:
                recom_score.append(i[1])

#### 抽样计算覆盖率

In [ ]:
tf.reset_default_graph()

#由于电脑以及tensorflow框架的局限性，随机取100个用户进行推荐，取每个用户推荐的前10个论文
all_dic={}
for i in range(0,10):
    user_id=random.sample(user_have,1)[0]
    #查询用户名所对应索引
    index=[user_have.index(user_id)]

    #用户对这个用户对所有电影的预测评分
    session1 = tf.Session()
    #expand_dims扩大指定维数 原来values[0][0]为（20，）的数组 变为(1,20)
    u1_dim = tf.expand_dims(values[0][index[0]], 0)
    
    #matmul a、b矩阵相乘 transpose_b：如果 True,b 在乘法之前转置
    u1_all = tf.matmul(u1_dim, values[1],transpose_b=True)

    #计算用户对所有商品的排序
    result_1 = session1.run(u1_all)
    
    #squeeze 将结果矩阵的第一维去掉，去掉外面一个框
    p = np.squeeze(result_1)
    
    #argsort 返回数组值从小到大的索引值
    #把大小除了最大五个以外的p里的其他索引对应的值都设为0
    p[np.argsort(p)[:-5000]] = 0

    #遍历p中的全部索引，只要值不为0，就输出索引和对应的索引值
    article_rec=[]
    score=[]
    for index in range(len(p)):
        if p[index] != 0:
            score.append([index,p[index]])
            article_rec.append(article_have[index])
            
            
    #转为数据框
    score_df=pd.DataFrame(score)

    #按顺序进行排序
    score_df2=score_df.sort_values(by=1, ascending=False)

    #筛选出这个用户感兴趣的分类号，四舍五入按比例选出前200个论文
    co=list(user_classcode[user_id].keys())
    co_sum=np.sum(list(user_classcode[user_id].values()))
    co_wi=[]
    for c in co:
        co_wi.append(int(round(user_classcode[user_id][c]/co_sum)))
    ar_incode=[]
    for c in co:
        flag=0
        for ar_index in list(score_df2[0].values):
            if article_have[ar_index] in article_classfy[c]:
                ar_incode.append(article_have[ar_index])
                flag+=1
                if flag>=200*co_wi[co.index(c)]:
                    break
                
                
    recom_article_id=[]
    for ar in ar_incode[0:15]:
        if ar in all_article_df['article_id'].values:
            recom_article_id.append(ar)
                
    all_dic[user_id]={}
    all_dic[user_id]['recom_article_id']=recom_article_id

In [32]:
#在测试过程中推荐的200个文献列表
f=open('随机抽取的10位用户推荐的117篇文献.txt','r')
a=f.read()
all_dic=eval(a)
f.close()

In [33]:
all_dic

{'3dec9e7a8c54251c': {'recom_article_id': ['ggyzl201807014',
   'zwjl201721039',
   'Y3262432',
   'kxdb-kxjy201517166',
   'zgfdc-xx201910006',
   'D767640',
   'syjlr201018104',
   'Y707824',
   'D01390515',
   'whqcgydxxb201505027',
   'dzjsjjgl201904012',
   'kjyj201110007',
   'kjzfxd-kxyzb201106198',
   'Y3496684',
   'jjwtts200401004']},
 '56668dab9978ec85': {'recom_article_id': ['Y2192158',
   '9516403',
   'xnmzxyxb201303013',
   'D01019667']},
 '5dcb0a55de97c649': {'recom_article_id': ['hljkjxx201010236',
   'shkx201705007',
   'D01702232']},
 '71aa6d67367afed6': {'recom_article_id': ['ycsfxyxb-rwsh201901003',
   'zgzckjs201507014',
   'fzbl201901188',
   'zgxlwszz200301014',
   'fzbl201901188']},
 '8626ac1c132554c5': {'recom_article_id': ['fskxjsxyxb201904002',
   'jtysxtgcyxx201901028',
   'xzqykj201413121',
   'D01058974',
   'Y2817730',
   'cxjs201504016',
   'D01078530',
   'txsj201909119',
   'qcsyjs201714058',
   'gckz201901015',
   'zljxysgjxh201802009',
   'tdbzsj201

In [35]:
#计算覆盖率
all_art=[]
for user in all_dic.keys():
    all_art+=all_dic[user]['recom_article_id']

#10个用户推荐100个文献中可视为无重复推荐100*116/117个文献
100*len(set(all_art))/len(all_art)/100

0.9914529914529915

### 论文部分数据计算

In [ ]:
sum=0
for code in article_classfy.keys():
    sum+=len(article_classfy[code])
#计算论文标签权重 qi/q总
w_code={}
for code in article_classfy.keys():
    w_code[code]=len(article_classfy[code])/sum

In [ ]:
#计算用户对某个标签的权重，即(此用户对这个标签的互动次数/此用户互动总次数)*此标签的权重
w_user={}
for user in user_classcode.keys():
    c_sum=np.sum(list(user_classcode[user].values()))
    for c in user_classcode[user]:
        if user in w_user.keys():
            w_user[user][c]=(user_classcode[user][c]/c_sum)*w_code[c]
        else:
            w_user[user]={}
            w_user[user][c]=(user_classcode[user][c]/c_sum)*w_code[c]

In [ ]:
#字典存储
#生成的字典存成txt的方法
f=open('用户标签权重.txt','w')
f.write(str(w_user))
f.close()

#字典存储
#生成的字典存成txt的方法
f=open('论文标签权重.txt','w')
f.write(str(w_code))
f.close()